In [ ]:
import torch
import torchlensmaker as tlm

surface = tlm.SphereC(1.0, C=0.2)

source = tlm.PointSourceAtInfinity2D(8.0, sampler_pupil=tlm.LinspaceSampler1D(50))
data = source(tlm.default_input(2))

t, normals, valid, dfk_out, ifk_out = surface(data.P, data.V, data.dfk, data.ifk)

# tlmviewer

scene = tlm.new_scene("2D")
cp = data.P + t.unsqueeze(-1)*data.V
scene["data"].append(tlm.render_rays(data.P, data.P + 2*data.V, 0))
scene["data"].append(tlm.render_points(cp, "white", 0.05))
scene["data"].append(tlm.render_arrows(cp, normals))
tlm.display_scene(scene)